# PREVIOUS WORK

In [18]:
# Start Python Imports
import math, time, random, datetime
from math import sqrt
# Data Manipulation
import numpy as np
import pandas as pd

# Visualization 
import matplotlib.pyplot as plt
import missingno
import seaborn as sns
plt.style.use('seaborn-whitegrid')

# Preprocessing
from scipy.linalg import svd
from sklearn.feature_selection import SelectKBest

from fancyimpute import IterativeImputer

# Modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score, GridSearchCV
from sklearn.preprocessing import MinMaxScaler

# Load data
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [19]:
def mice_imputation(df):
    XY_incomplete = df

    n_imputations = 5
    XY_completed = []
    for i in range(n_imputations):
        imputer = IterativeImputer(n_iter=5, sample_posterior=True, random_state=i)
        XY_completed.append(imputer.fit_transform(XY_incomplete))
    
    
    return pd.DataFrame(data=XY_incomplete, columns=XY_incomplete.columns, index=XY_incomplete.index)

def fill_missing_or_nan_values(df):
    fill_with = 0
    
    # Get the most common element by using size(),
    # which returns the element and how common it is
    for column in df:
        
        # Check if the column is an object, float64 or int64
        is_it_float = (df[column].dtype == np.float64)
        is_it_int = (df[column].dtype == np.int64)
        
        # If it is an object,
        # find the most common element and fill missing and NaN values
        if(not is_it_float and not is_it_int):
            fill_with = df[column].mode().item()
                    
        # If it is either a float64 or int64,
        # then calculate the mean and fill missing and NaN values
        else:
            if is_it_float:
                fill_with = np.nanmean(df[column], dtype=np.float64)
            if is_it_int:
                fill_with = np.nanmean(df[column], dtype=np.int64)
        
        # Fill the values in our dataset
        df[column] = df[column].fillna(fill_with)
        fill_with = 0

def data_engineering(train, test):
    # Concatenate all of data
    cc_data = pd.concat([train, test])
    cc_data = cc_data.drop(['Id', 'SalePrice','Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature'], axis=1)
    
    # Get the SalePrice as the natural logarithm
    train["SalePrice"] = np.log1p(train["SalePrice"])
    y = train['SalePrice']
    
    # One-Hot encode all data
    cc_data = pd.get_dummies(cc_data, prefix_sep='_')
    
    #MICE Imputation
    cc_data = mice_imputation(cc_data)
    
    # Fill missing or NaN values
    #fill_missing_or_nan_values(cc_data)
    
    # Slice data, train.shape[0] is the observations
    # 1) from start to middle of observations
    # 2) from middle of observations to end
    X_train = cc_data[:train.shape[0]]
    X_test = cc_data[train.shape[0]:]
    
    return X_train,X_test,y

# X is dataframe, y is output, m is how many features you want selected
# returns array of highest scoring features
def feature_selection(X, y, m):
    # Data is standardized here, minus mean and divided by standard deviation
    # The correlation between each regressor and the target is computed
    # It is converted to an F score then to a p-value, which is returned
    f_regression = lambda X,y : sklearn.feature_selection.f_regression(X,y,center=False)

    # removes all but the  highest scoring features
    featureSelector = SelectKBest(score_func=f_regression,k=m)
    featureSelector.fit(X,y)
    high_score_arr = [X.columns[1+zero_based_index] for zero_based_index in list(featureSelector.get_support(indices=True))]
    
    return high_score_arr

# RANDOM FOREST WORK

In [20]:
def random_forest_prediction(X_train,X_test,y_real):
    gs = GridSearchCV(
        estimator=RandomForestRegressor(),
        param_grid={
            'max_depth': [3, None],
            'n_estimators': (10, 30, 50, 100, 200, 400, 600, 800, 1000)
        }, cv=10, n_jobs=-1, scoring='neg_mean_squared_error'
    )
    model = gs.fit(X_train,y_real)
    pred = model.predict(X_test)
    score = sqrt(-model.best_score_)
    
    # return all predictions and mean of all cross validated scores
    return pred, score

df_train,df_test,y = data_engineering(train,test)
#selected_features = feature_selection(df_train, y, 50)

#pred,score = random_forest_prediction(df_train[selected_features], df_test[selected_features], y)

/Users/ahmedmagdy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [21]:
def findMissingValues(df, columns):
    """
    Finds number of rows where certain columns are missing values.
    ::param_df:: = target dataframe
    ::param_columns:: = list of columns
    """
    missingValues = {}
    print("Number of missing or NaN values for each column:")
    dfLength = len(df)
    for column in columns:
        totalColumnValues = df[column].value_counts().sum()
        missingValues[column] = dfLength-totalColumnValues
    return missingValues

callMissingValues = findMissingValues(df_train, columns=df_train.columns)
callMissingValues

Number of missing or NaN values for each column:


{'1stFlrSF': 0,
 '2ndFlrSF': 0,
 '3SsnPorch': 0,
 'BedroomAbvGr': 0,
 'BsmtFinSF1': 0,
 'BsmtFinSF2': 0,
 'BsmtFullBath': 0,
 'BsmtHalfBath': 0,
 'BsmtUnfSF': 0,
 'EnclosedPorch': 0,
 'Fireplaces': 0,
 'FullBath': 0,
 'GarageArea': 0,
 'GarageCars': 0,
 'GarageYrBlt': 81,
 'GrLivArea': 0,
 'HalfBath': 0,
 'KitchenAbvGr': 0,
 'LotArea': 0,
 'LotFrontage': 259,
 'LowQualFinSF': 0,
 'MSSubClass': 0,
 'MasVnrArea': 8,
 'MiscVal': 0,
 'MoSold': 0,
 'OpenPorchSF': 0,
 'OverallCond': 0,
 'OverallQual': 0,
 'PoolArea': 0,
 'ScreenPorch': 0,
 'TotRmsAbvGrd': 0,
 'TotalBsmtSF': 0,
 'WoodDeckSF': 0,
 'YearBuilt': 0,
 'YearRemodAdd': 0,
 'YrSold': 0,
 'BldgType_1Fam': 0,
 'BldgType_2fmCon': 0,
 'BldgType_Duplex': 0,
 'BldgType_Twnhs': 0,
 'BldgType_TwnhsE': 0,
 'BsmtCond_Fa': 0,
 'BsmtCond_Gd': 0,
 'BsmtCond_Po': 0,
 'BsmtCond_TA': 0,
 'BsmtExposure_Av': 0,
 'BsmtExposure_Gd': 0,
 'BsmtExposure_Mn': 0,
 'BsmtExposure_No': 0,
 'BsmtFinType1_ALQ': 0,
 'BsmtFinType1_BLQ': 0,
 'BsmtFinType1_GLQ': 0,
 

In [22]:
# To convert prediction of SalePrice into the actual value, we take exponential value
print(np.exp(pred))
print(score)

NameError: name 'pred' is not defined

In [ ]:
'''
# s can be array of functions for different models to run
def kfold_cv_models(k,s):
    best_models_test_error = 0

    for fold in range(k):
        # we have k partitions
        # let df_train and df_test be the next partition
        for model in range(s):
            # run next model with current k with df_train
            # get test error of df_test
            if this_models_test_error < best_models_test_error:
                best_models_test_error = this_models_test_error
    return best_models_test_error
'''